In [ ]:
%matplotlib inline

# Fetching data from the IOOS CSW catalog

This notebook shows a typical workflow to query a [Catalog Service for the Web (CSW)](https://en.wikipedia.org/wiki/Catalog_Service_for_the_Web) and create a request for data endpoints that are suitable for download.

The first step is to create the data filter based on the **geographical** region bounding box, the **time** span, and the CF **variable** standard name.

### Region: west coast

In [ ]:
min_lon, max_lon = -123, -121
min_lat, max_lat = 36, 40

bbox = [min_lon, min_lat, max_lon, max_lat]
crs = "urn:ogc:def:crs:OGC:1.3:CRS84"

### Temporal range: past week

In [ ]:
from datetime import datetime

start, stop = datetime(2019, 7, 1), datetime(2019, 7, 14)

### Surface velocity CF names

In [ ]:
cf_names = [
    "surface_northward_sea_water_velocity",
    "surface_eastward_sea_water_velocity",
]

In [ ]:
print(
    f"""
*standard_names*: {cf_names}
*start and stop dates*: {start} to {stop}
*bounding box*:{bbox}
*crs*: {crs}"""
)

Now it is possible to assemble a [OGC Filter Encoding (FE)](http://www.opengeospatial.org/standards/filter) for the search using `owslib.fes`\*. Note that the final result is only a list with all the filtering conditions.

\* OWSLib is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models.

In [ ]:
from owslib import fes


def fes_date_filter(start, stop, constraint="overlaps"):
    start = start.strftime("%Y-%m-%d %H:00")
    stop = stop.strftime("%Y-%m-%d %H:00")
    if constraint == "overlaps":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
    elif constraint == "within":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
    else:
        raise NameError(f"Unrecognized constraint {constraint}")
    return begin, end

### Create the filter

In [ ]:
kw = dict(wildCard="*", escapeChar="\\", singleChar="?", propertyname="apiso:AnyText")

or_filt = fes.Or([fes.PropertyIsLike(literal=("*%s*" % val), **kw) for val in cf_names])

# Exclude GNOME returns.
not_filt = fes.Not([fes.PropertyIsLike(literal="*GNOME*", **kw)])

begin, end = fes_date_filter(start, stop)
bbox_crs = fes.BBox(bbox, crs=crs)
filter_list = [fes.And([bbox_crs, begin, end, or_filt, not_filt])]

In [ ]:
def get_csw_records(csw, filter_list, pagesize=10, maxrecords=1000):
    """Iterate maxrecords/pagesize times until the requested
    value in maxrecords is reached."""
    from owslib.fes import SortBy, SortProperty

    sortby = SortBy([SortProperty("dc:title", "ASC")])
    csw_records = {}
    startposition = 0
    nextrecord = getattr(csw, "results", 1)
    while nextrecord != 0:
        csw.getrecords2(
            constraints=filter_list,
            startposition=startposition,
            maxrecords=pagesize,
            sortby=sortby,
        )
        csw_records.update(csw.records)
        if csw.results["nextrecord"] == 0:
            break
        startposition += pagesize + 1  # Last one is included.
        if startposition >= maxrecords:
            break
    csw.records.update(csw_records)

In [ ]:
from owslib.csw import CatalogueServiceWeb

endpoint = "https://data.ioos.us/csw"

csw = CatalogueServiceWeb(endpoint, timeout=60)
get_csw_records(csw, filter_list, pagesize=10, maxrecords=1000)

records = "\n".join(csw.records.keys())

In [ ]:
print(f"Found {len(csw.records.keys())} records.\n")
for key, value in list(csw.records.items()):
    print(f"[{value.title}]: {key}")

Let us check the 6 km resolution metadata record we found above.

In [ ]:
value = csw.records["edu.ucsd.cordc:202004230600siohfruwlsrtvuswc6km"]

print(value.abstract)

In [ ]:
attrs = [attr for attr in dir(value) if not attr.startswith("_")]
nonzero = [attr for attr in attrs if getattr(value, attr)]
nonzero

### What is in there?

In [ ]:
value.subjects

### Is it up-to-date?

In [ ]:
value.modified

### The actual bounding box of the data

In [ ]:
bbox = (value.bbox.minx, value.bbox.miny, value.bbox.maxx, value.bbox.maxy)
bbox

### Now we can "sniff" the URLs with *geolinks*

In [ ]:
from geolinks import sniff_link

for ref in value.references:
    if ref["scheme"] == "OPeNDAP:OPeNDAP":
        url = ref["url"]  # save the opendap for later
    geolink = sniff_link(ref["url"])
    msg = f"geolink: {geolink}\nscheme: {ref['scheme']}\n" f"URL: {ref['url']}\n"
    print(msg)

For a detailed description of what those `geolink` results mean check the [lookup](https://github.com/OSGeo/Cat-Interop/blob/master/LinkPropertyLookupTable.csv) table.
There are Web Coverage Service (WCS), Web Map Service (WMS),
direct links, and OPeNDAP services available.

We can use any of those to obtain the data but the easiest one to explore interactively is the open OPeNDAP endpoint.

In [ ]:
import xarray as xr

ds = xr.open_dataset(url)
ds

### Select "last week" data

In [ ]:
from datetime import date, timedelta

yesterday = date.today() - timedelta(days=1)
sliced = ds.sel(time=yesterday.strftime("%Y-%m-%d 00:00:00"))
sliced

### Compute the speed while masking invalid values

In [ ]:
import numpy.ma as ma

u = sliced["u"].data
v = sliced["v"].data

lon = sliced.coords["lon"].data
lat = sliced.coords["lat"].data
time = sliced.coords["time"].data

u = ma.masked_invalid(u)
v = ma.masked_invalid(v)

This cell is only a trick to show all quiver arrows with the same length,
for visualization purposes,
and indicate the vector magnitude with colors instead.

In [ ]:
import numpy as np
from oceans.ocfis import spdir2uv, uv2spdir

angle, speed = uv2spdir(u, v)
us, vs = spdir2uv(np.ones_like(speed), angle, deg=True)

### And now we are ready to create the plot

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy import feature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER

LAND = feature.NaturalEarthFeature(
    "physical", "land", "10m", edgecolor="face", facecolor="lightgray",
)

sub = 2
dx = dy = 0.5
center = -122.416667, 37.783333  # San Francisco.
bbox = lon.min(), lon.max(), lat.min(), lat.max()

In [ ]:
def plot_hfradar():
    fig, (ax0, ax1) = plt.subplots(
        ncols=2, figsize=(20, 20), subplot_kw=dict(projection=ccrs.PlateCarree())
    )

    ax0.set_extent(bbox)
    cs = ax0.pcolor(lon, lat, ma.masked_invalid(speed), shading="auto")
    gl = ax0.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    cbar = fig.colorbar(cs, ax=ax0, shrink=0.45, extend="both")
    cbar.ax.set_title(r"speed m s$^{-1}$", loc="left")

    ax0.add_feature(LAND, zorder=0, edgecolor="black")
    ax0.set_title(f"{value.title}\n{ds['time'].values}")

    ax1.set_extent(
        [center[0] - dx - dx, center[0] + dx, center[1] - dy, center[1] + dy]
    )
    q = ax1.quiver(
        lon[::sub],
        lat[::sub],
        us[::sub, ::sub],
        vs[::sub, ::sub],
        speed[::sub, ::sub],
        scale=30,
    )
    ax1.quiverkey(q, 0.5, 0.85, 1, r"1 m s$^{-1}$", coordinates="axes")
    gl = ax1.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    ax1.add_feature(LAND, zorder=0, edgecolor="black")
    ax1.set_title("San Francisco Bay area")

In [ ]:
plot_hfradar()

### Try to create your own filter and enjoy data!

```python
filter_list = [
    fes.And(
        [
            bbox_crs,
            begin, end,
            or_filt, not_filt,
        ]
    )
]
```